Перед нами стоит задача защиты данных клиентов страховой компании «Хоть потоп». 

Важно, чтобы при защите персональных данных качество моделей машинного обучения не ухудшилось. 

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [5]:
df.duplicated().sum()

153

In [6]:
df = df.drop_duplicates().reset_index(drop=True)

In [7]:
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Данные загружены, изучены, с форматом ознакомилась <br>
Найденные дубли были удалены, пропусков нет - данные чистые и готовы к преобразованию <br>

## Алгоритм преобразования

Обозначения:

$X$ — матрица признаков (нулевой столбец состоит из единиц)

$y$ — вектор целевого признака

$P$ — матрица, на которую умножаются признаки

$w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$
Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$
Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$
Ответ: Не изменится

Обоснование:

$$
w = (X^T X)^{-1} X^T y
$$
Преобразование матрицы Х$$
w' = ((XP)^T (XP))^{-1} (XP)^T y =
$$

$$
= (P^T X^T X P)^{-1}  (XP)^T y =
$$$$
= (P^T (X^TX) P)^{-1} (XP)^T y = 
$$$$
= (P^T ((X^TX) P))^{-1} P^T X^T y =
$$$$
= ((X^TX)P)^{-1} (((P^T))^{-1}P^T) X^T y = ((X^TX)P)^{-1} E X^T y =
$$$$
= P^{-1}(X^TX)^{-1} X^Ty
$$
Предсказание$$
a = Xw = 
$$$$
= X (X^T X)^{-1} X^T y
$$Предсказане на преобразованных данных$$
a' = XP P^{-1}(X^TX)^{-1} X^Ty = XE(X^TX)^{-1} X^Ty
$$$$
= X (X^T X)^{-1} X^T y
$$

**Ответ:**

1) Создаём квадратную матрицу A из рандомных чисел с помощью np.random.randint()<br>
2) Проверяем созданную матрицу на обратимость с помощью np.linalg.inv() <br>
3) Матрица признаков x квадратная матрица = зашифрованные данные <br>
4) Для дешифрования используем умножение матрицы зашифрованных параметров на обратную матрицу <br>
5) Проверяем качество расшифровки <br>
6) При нарушениях работы алгоритма - изучаем их и исправляем <br>

## Умножение матриц

**Обоснование:** 

1) Матрица x обратимая квадратная матрица = зашифрованные данные клиента <br>
2) Ключ шифра - обратная матрица <br>
3) Данный подход не должен ухудшить качество модели линейной регрессии <br>

## Проверка алгоритма

In [8]:
features = df.drop('Страховые выплаты', axis = 1)
target = df['Страховые выплаты']

In [9]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [10]:
state = np.random.RandomState(12345)
rand_matrix = state.randint(0, 100, size = (4, 4))
rand_matrix

array([[98, 29,  1, 36],
       [41, 34, 29,  1],
       [59, 14, 91, 80],
       [73, 11, 77, 10]])

In [11]:
inv_matrix = np.linalg.inv(rand_matrix)
res_matrix = pd.DataFrame(rand_matrix @ inv_matrix, columns = [0,1,2,3])
res_matrix

,0,1,2,3
0,1.000000e+00,1.526557e-16,-3.469447e-17,6.938894e-17
1,-9.540979e-18,1.000000e+00,-2.602085e-17,5.898060e-17
2,-9.714451e-17,5.551115e-17,1.000000e+00,2.775558e-16
3,8.673617e-18,4.857226e-17,1.075529e-16,1.000000e+00


In [12]:
mod_features_train = features_train @ rand_matrix
mod_features_valid = features_valid @ rand_matrix

In [13]:
model = LinearRegression()
model.fit(features_train, target_train)
model_predicted = model.predict(features_valid)

In [14]:
encoded_model = LinearRegression()
encoded_model.fit(mod_features_train, target_train)
encoded_models_predicted = encoded_model.predict(mod_features_valid)

In [15]:
r2score = r2_score(target_valid, model_predicted)
encoded_models_r2_score = r2_score(target_valid, encoded_models_predicted)
mse_normal = mean_squared_error(target_valid, model_predicted)
mse_encoded = mean_squared_error(target_valid, encoded_models_predicted)

print('R2 нешифрованной даты:', r2score, 'MSE:', mse_normal)
print('R2 шифрованной даты:', encoded_models_r2_score, 'MSE:', mse_encoded)

R2 нешифрованной даты: 0.42307727492147296 MSE: 0.11955009374099915
R2 шифрованной даты: 0.42307727492236724 MSE: 0.11955009374081381


Провела обучение модели линейной регрессии на исходном датайфрейме и закодированном <br>
Выбрав в качестве метрик R2 и MSE, получила удовлетворяющее качество показателей <br>

In [16]:
features_train

,Пол,Возраст,Зарплата,Члены семьи
4460,0,28.0,54500.0,2
3773,1,38.0,50700.0,1
4561,0,24.0,28000.0,1
1385,0,47.0,38800.0,0
724,0,40.0,32500.0,1
...,...,...,...,...
3497,1,23.0,28200.0,1
3492,0,30.0,33700.0,1
2177,1,20.0,28900.0,1
3557,1,33.0,38800.0,1


In [17]:
returned_features_train = mod_features_train @ inv_matrix
returned_features_valid = mod_features_valid @ inv_matrix
returned_features_train

,0,1,2,3
4460,-5.691794e-12,28.0,54500.0,2.000000e+00
3773,1.000000e+00,38.0,50700.0,1.000000e+00
4561,-4.372445e-12,24.0,28000.0,1.000000e+00
1385,-3.713003e-12,47.0,38800.0,6.316472e-12
724,-2.552720e-12,40.0,32500.0,1.000000e+00
...,...,...,...,...
3497,1.000000e+00,23.0,28200.0,1.000000e+00
3492,-2.889709e-12,30.0,33700.0,1.000000e+00
2177,1.000000e+00,20.0,28900.0,1.000000e+00
3557,1.000000e+00,33.0,38800.0,1.000000e+00


**~~Шалость~~ Шифрование удалось.** <br> Судя по всему, важно, чтобы матрица-шифровщик была квадратной и имела обратную с соответствующим количеством столбцов в исходной матрице.

In [18]:
returned_features_train.columns = ['Пол', 'Возраст', 'Зарплата', 'Члены семьи'] 
returned_features_train = round(returned_features_train[['Пол', 'Возраст', 'Зарплата', 'Члены семьи']], 2)
features_train = round(features_train[['Пол', 'Возраст', 'Зарплата', 'Члены семьи']], 2)


for i in ['Пол', 'Возраст', 'Зарплата', 'Члены семьи']:
    print('Совпдадает ли признак', i, returned_features_train[i].isin(features_train[i]).unique())

Совпдадает ли признак Пол [ True]
Совпдадает ли признак Возраст [ True]
Совпдадает ли признак Зарплата [ True]
Совпдадает ли признак Члены семьи [ True]


**Выводы:**

1) Предложенный алгоритм шифра - работает <br>
2) Данные удалось зашифровать <br>
3) Модели с исходными и зашифрованными данные показали практические идентичные показатели взятых метрик качества - R2 и MSE <br>